In [2]:
from fastai.text.all import *
from fastai.callback.progress import CSVLogger

In [19]:
model_path = Path('data/model')
model_path.mkdir(exist_ok=True)
path = Path('data/doc_small')
lang = 'id'

In [4]:
bs=128

In [5]:
tok = SentencePieceTokenizer(lang=lang)

In [6]:
dblock = DataBlock(blocks=TextBlock.from_folder(path, is_lm=True, tok=tok),
                   get_items=get_files,
                   splitter=RandomSplitter(valid_pct=0.1, seed=42),
                  )

dls = dblock.dataloaders(path, path=path, bs=bs, num_workers=18)

In [7]:
dls.show_batch()

,text,text_
0,"▁xxbos ▁ spongebob ▁squarepants ▁( musim ▁ke - 8) ▁xxmaj ▁musim ▁kedelapan ▁"" spongebob ▁squarepants "" ▁dimulai ▁pada ▁26 ▁xxmaj ▁maret ▁2011 ▁dengan ▁"" a ▁xxmaj ▁f ri end ly ▁xxmaj ▁game "" ▁dan ▁"" or al ▁xxmaj ▁report "" ▁dan ▁berakhir ▁pada ▁6 ▁xxmaj ▁desember ▁2012 ▁dengan ▁"" it ' s ▁ a ▁ spongebob ▁xxmaj ▁christ mas !"" . ▁xxmaj ▁musim ▁ini ▁ter diri ▁dari ▁26 ▁episode . ▁xxmaj ▁beberapa","▁ spongebob ▁squarepants ▁( musim ▁ke - 8) ▁xxmaj ▁musim ▁kedelapan ▁"" spongebob ▁squarepants "" ▁dimulai ▁pada ▁26 ▁xxmaj ▁maret ▁2011 ▁dengan ▁"" a ▁xxmaj ▁f ri end ly ▁xxmaj ▁game "" ▁dan ▁"" or al ▁xxmaj ▁report "" ▁dan ▁berakhir ▁pada ▁6 ▁xxmaj ▁desember ▁2012 ▁dengan ▁"" it ' s ▁ a ▁ spongebob ▁xxmaj ▁christ mas !"" . ▁xxmaj ▁musim ▁ini ▁ter diri ▁dari ▁26 ▁episode . ▁xxmaj ▁beberapa ▁xxup"
1,"▁leo ▁xxmaj ▁david son ▁bekerja ▁sama ▁dengan ▁pri mata ▁yang ▁di latih ▁untuk ▁misi ▁ruang ▁angkasa . ▁xxmaj ▁re kan ▁kerja ▁sim ian ▁f a vo rit nya ▁adalah ▁se ekor ▁sim pan se ▁bernama ▁xxmaj ▁pericles . ▁xxmaj ▁dengan ▁badai ▁el ek tro m a g ne tik ▁mem atikan ▁mendekati ▁stasiun , ▁ruang ▁kecil ▁p od ▁dik emudi kan ▁oleh ▁xxmaj ▁pericles ▁digunakan ▁untuk ▁m enyelidik i ▁badai . ▁xxmaj","▁xxmaj ▁david son ▁bekerja ▁sama ▁dengan ▁pri mata ▁yang ▁di latih ▁untuk ▁misi ▁ruang ▁angkasa . ▁xxmaj ▁re kan ▁kerja ▁sim ian ▁f a vo rit nya ▁adalah ▁se ekor ▁sim pan se ▁bernama ▁xxmaj ▁pericles . ▁xxmaj ▁dengan ▁badai ▁el ek tro m a g ne tik ▁mem atikan ▁mendekati ▁stasiun , ▁ruang ▁kecil ▁p od ▁dik emudi kan ▁oleh ▁xxmaj ▁pericles ▁digunakan ▁untuk ▁m enyelidik i ▁badai . ▁xxmaj ▁pericles"
2,"▁umumnya ▁terjal ▁dan ▁langsung ▁di ba tasi ▁oleh ▁laut ▁dalam ▁yang ▁ cenderung ▁merupakan ▁jalur ▁se s ar . ▁xxmaj ▁berikut ▁beberapa ▁peta ▁kuno ▁tentang ▁"" pulau ▁xxmaj ▁selayar "" ▁yang ▁dibuat ▁pada ▁tahun ▁188 6 ▁dan ▁1920 ▁xxbos ▁xxup ▁smp ▁xxmaj ▁negeri ▁1 ▁xxmaj ▁kual a ▁xxmaj ▁tungkal ▁xxup ▁smp ▁xxmaj ▁negeri ▁1 ▁xxmaj ▁kual a ▁xxmaj ▁tungkal ▁adalah ▁sebuah ▁sekolah ▁menengah ▁pertama ▁yang ▁ter letak ▁di ▁xxmaj ▁jalan ▁xxup ▁h","▁terjal ▁dan ▁langsung ▁di ba tasi ▁oleh ▁laut ▁dalam ▁yang ▁ cenderung ▁merupakan ▁jalur ▁se s ar . ▁xxmaj ▁berikut ▁beberapa ▁peta ▁kuno ▁tentang ▁"" pulau ▁xxmaj ▁selayar "" ▁yang ▁dibuat ▁pada ▁tahun ▁188 6 ▁dan ▁1920 ▁xxbos ▁xxup ▁smp ▁xxmaj ▁negeri ▁1 ▁xxmaj ▁kual a ▁xxmaj ▁tungkal ▁xxup ▁smp ▁xxmaj ▁negeri ▁1 ▁xxmaj ▁kual a ▁xxmaj ▁tungkal ▁adalah ▁sebuah ▁sekolah ▁menengah ▁pertama ▁yang ▁ter letak ▁di ▁xxmaj ▁jalan ▁xxup ▁h ."
3,"▁xxmaj ▁negeri ▁1 ▁xxmaj ▁kual a ▁xxmaj ▁tungkal ▁ter letak ▁di ▁xxmaj ▁ibu ▁xxmaj ▁kota ▁xxmaj ▁kabupaten ▁xxmaj ▁t anjung ▁xxmaj ▁jabung ▁xxmaj ▁barat , ▁namun ▁jauh ▁dari ▁kera m a ian ▁pasar ▁serta ▁ a rus ▁lalu ▁lintas ▁ram a i , ▁jara k ▁dari ▁sekolah ▁ke ▁pusat ▁pasar ▁kurang ▁lebih ▁1 ▁xxmaj ▁km ▁dan ▁jara k ▁dari ▁ru a s ▁jalan ▁raya ▁sekitar ▁150 ▁xxup ▁m . ▁xxmaj ▁berdasarkan ▁data","▁negeri ▁1 ▁xxmaj ▁kual a ▁xxmaj ▁tungkal ▁ter letak ▁di ▁xxmaj ▁ibu ▁xxmaj ▁kota ▁xxmaj ▁kabupaten ▁xxmaj ▁t anjung ▁xxmaj ▁jabung ▁xxmaj ▁barat , ▁namun ▁jauh ▁dari ▁kera m a ian ▁pasar ▁serta ▁ a rus ▁lalu ▁lintas ▁ram a i , ▁jara k ▁dari ▁sekolah ▁ke ▁pusat ▁pasar ▁kurang ▁lebih ▁1 ▁xxmaj ▁km ▁dan ▁jara k ▁dari ▁ru a s ▁jalan ▁raya ▁sekitar ▁150 ▁xxup ▁m . ▁xxmaj ▁berdasarkan ▁data ▁yang"
4,") ▁mengenai ▁pemilihan ▁umum ▁xxmaj ▁presiden ▁xxmaj ▁amerika ▁xxmaj ▁serikat ▁2 ▁xxrep ▁3 ▁0 ▁ , ▁dan ▁"" o ber ge f ell ▁v . ▁xxmaj ▁ho d ge s "" ▁(201 5) ▁tentang ▁pernikahan ▁sesam a ▁jenis . ▁xxmaj ▁amendemen ▁ini ▁memb a tasi ▁tindakan ▁semua ▁pejabat ▁negara , ▁termasuk ▁mereka ▁yang ▁ber tindak ▁atas ▁nama ▁seorang ▁pejabat . ▁xxmaj ▁ayat ▁pertama ▁amendemen ▁ini ▁meng andung ▁beberapa ▁klausa , ▁seperti ▁xxmaj","▁mengenai ▁pemilihan ▁umum ▁xxmaj ▁presiden ▁xxmaj ▁amerika ▁xxmaj ▁serikat ▁2 ▁xxrep ▁3 ▁0 ▁ , ▁dan ▁"" o ber ge f ell ▁v . ▁xxmaj ▁ho d ge s "" ▁(201 5) ▁tentang ▁pernikahan ▁sesam a ▁jenis . ▁xxmaj ▁amendemen ▁ini ▁memb a tasi ▁tindakan ▁semua ▁pejabat ▁negara , 

In [8]:
len(dls.train), len(dls.valid)

(15, 2)

In [15]:
#learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=False, 
#                               metrics=[accuracy, Perplexity()]).to_native_fp16()
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=False, 
                               metrics=[accuracy, Perplexity()])
learn.path = model_path

In [16]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [17]:
cbs=[CSVLogger(fname='history_de_wikitext.csv')]

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, cbs=cbs, moms=(0.8, 0.7, 0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,7.249102,6.810330,0.124009,907.170471,01:06
1,7.240525,7.713139,0.015674,2237.553955,02:42


In [ ]:
lm_fns = [model_path/f'{lang}_wikitext', model_path/f'{lang}_wikitext_vocab.pkl']

In [ ]:
#learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.save(lm_fns[0], with_opt=False)

In [ ]:
with open(lm_fns[1], 'wb') as f:
      pickle.dump(learn.dls.vocab, f)